For each year we will precalculate levels of gas leaks per person for each geoid. Then we will group geoids by levels and create geolayers. So, each layer will have a group of geoid with ~ same level of gas leaks per person

In [18]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np
import re

mapbox_access_token = "pk.eyJ1IjoibWlzaGtpY2UiLCJhIjoiY2s5MG94bWRoMDQxdjNmcHI1aWI1YnFkYyJ9.eFsHqEMYY7qxa0Pb9USCtQ"

In [19]:
dfg = gpd.read_file('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/2010 Census Tracts with geoid.geojson')
dfg['geoid'] = dfg.block_fips.str[:11]
dfg = dfg[['geoid', 'ntaname', 'boro_name', 'shape_area', 'geometry']]
dfg.columns = map(str.lower, dfg.columns)
dfg = dfg.applymap(lambda s:s.lower() if type(s) == str else s)

In [20]:
dfg.head()

,geoid,ntaname,boro_name,shape_area,geometry
0,36085000900,west new brighton-new brighton-st. george,staten island,2497009.71359,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ..."
1,36061009800,turtle bay-east midtown,manhattan,1906016.35002,"MULTIPOLYGON (((-73.96433 40.75638, -73.96479 ..."
2,36061010000,turtle bay-east midtown,manhattan,1860938.37721,"MULTIPOLYGON (((-73.96802 40.75958, -73.96848 ..."
3,36061010200,midtown-midtown south,manhattan,1860992.70485,"MULTIPOLYGON (((-73.97124 40.76094, -73.97170 ..."
4,36061010400,midtown-midtown south,manhattan,1864600.37503,"MULTIPOLYGON (((-73.97446 40.76229, -73.97491 ..."


In [21]:
full_df = pd.read_csv('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/important_(used_in_app)/Merged_asc_fdny_data.csv')

In [23]:
full_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [24]:
full_df = full_df.dropna()

In [27]:
full_df[full_df.gas_leaks_per_person > 0.003310].describe()

,incident_year,geoid,gas_leaks_per_person,avg_assigned_fdny_engines#,avg_incident_duration,median_age,avg_houshold_size,nonfamily_housholds%,lonely_housholder%,lonely_housholder_over65%,housholders_grandparents_responsible_for_grandchildren%,unemployed%,median_houshold_income,mean_houshold_income,total_housing_units,occupied_housing_units%,vacant_housing_units%,not_us_citizen%
count,2889.000000,2.889000e+03,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000,2889.000000
mean,2015.771201,3.604885e+10,0.007172,2.062730,1392.592602,35.876843,2.542378,43.387317,35.088445,11.002267,2.224546,10.436622,57330.690550,82635.638629,1823.581516,90.168847,9.831326,14.606438
std,1.559281,2.503189e+07,0.020157,0.197013,378.556565,6.532454,0.526167,16.276726,13.111069,6.958534,3.071707,6.525277,36490.440626,57392.139721,1238.439008,7.634382,7.634398,7.352100
min,2013.000000,3.600500e+10,0.003310,0.714286,629.250000,15.700000,1.030000,0.000000,0.000000,0.000000,0.000000,0.000000,9053.000000,11652.000000,16.000000,16.100000,0.000000,0.000000
25%,2015.000000,3.604702e+10,0.003935,1.960000,1151.666667,31.700000,2.210000,31.300161,26.112760,6.688778,0.000000,5.700000,30893.000000,45552.000000,1043.000000,87.500000,5.200000,9.407070
50%,2016.000000,3.604711e+10,0.004911,2.060000,1338.250000,34.900000,2.550000,41.403126,33.333333,9.860724,1.454898,9.300000,46810.000000,65599.000000,1555.000000,91.600000,8.400000,13.370474
75%,2017.000000,3.606102e+10,0.006981,2.166667,1539.035714,38.900000,2.900000,53.057065,42.257742,13.907285,3.426791,13.900000,74712.000000,98881.000000,2281.000000,94.800000,12.500000,18.858308
max,2018.000000,3.608503e+10,0.750000,3.500000,4565.461538,84.300000,4.930000,100.000000,96.814672,82.526316,74.647887,78.300000,250001.000000,458625.000000,13033.000000,100.000000,83.900000,56.237817


<h3>Add bins column</h3>

In [28]:

bins_leaks_per_person = [
    "0-0.0015",
    "0.00151-0.003",
    "0.0031-0.0045",
    "0.00451-0.006",
    "0.0061-0.0075",
    "0.00751-0.009",
    "0.0091-10"
]

In [30]:
full_df['bins_leaks_per_person']=full_df['gas_leaks_per_person'].apply(lambda x: bins_leaks_per_person[0] if float(x) <=0.0015 else bins_leaks_per_person[1] if float(x) <=0.003 else bins_leaks_per_person[2] if float(x) <=0.0045 else bins_leaks_per_person[3] if float(x) <=0.006 else bins_leaks_per_person[4] if float(x) <=0.0075 else bins_leaks_per_person[5] if float(x) <=0.009 else bins_leaks_per_person[6] )

In [31]:
dfg['geoid']= dfg['geoid'].astype('int64')

<h3> Create geojson files according to bins</h3>

In [32]:
merged = dfg.merge(full_df, on='geoid' )

In [34]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 11538 entries, 0 to 11537
Data columns (total 23 columns):
 #   Column                                                   Non-Null Count  Dtype   
---  ------                                                   --------------  -----   
 0   geoid                                                    11538 non-null  int64   
 1   ntaname                                                  11538 non-null  object  
 2   boro_name                                                11538 non-null  object  
 3   shape_area                                               11538 non-null  object  
 4   geometry                                                 11538 non-null  geometry
 5   incident_year                                            11538 non-null  int64   
 6   gas_leaks_per_person                                     11538 non-null  float64 
 7   avg_assigned_fdny_engines#                               11538 non-null  float64 
 8   avg_inci

In [38]:
years = [2013 + x for x in range(6)]
for year in years:
    for bin in bins_leaks_per_person:
        print(bin)
        geo_layer = merged[(merged['bins_leaks_per_person'] == bin) & (merged['incident_year'] == year)] # geo

        try:
            geo_layer.to_file("C:/Users/mskac/machineLearning/GasLeakConEd/data/geolayers/{}.geojson".format(str(year)+'_'+bin), driver='GeoJSON')
        except:
            print('err')

0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
0-0.0015
0.00151-0.003
0.0031-0.0045
0.00451-0.006
0.0061-0.0075
0.00751-0.009
0.0091-10
